In [10]:
from tqdm import tqdm
import torch

device_str = 'cuda'
generator = torch.Generator(device=device_str)

In [11]:
import json

def read_data():
    with open('/kaggle/input/qa-persian/data.jsonl', 'r') as f:
        for line in f:
            yield json.loads(line)

In [12]:
from collections import Counter

texts = ''.join([(d['text'] or '') + (d['answer'] or '') for d in read_data()])
chars = Counter(texts)
chars = ['<s>'] + [c for c, cnt in chars.most_common() if cnt >= 1000]
ctoi = {c: i for i, c in enumerate(chars)}

In [13]:
conc = []
for i, d in enumerate(read_data()):
    conc += ['<s>']
    conc += list(d['text'] or '')
    conc += ['<s>']
    conc += list(d['answer'] or '')
    if i == 10000:
        break
X = []
for x in conc:
    if x in chars:
        X.append(ctoi[x])
X = torch.tensor(X, device=device_str)
X.shape

torch.Size([5156519])

In [14]:
window_len = 10
X_train = torch.concat([X[i:len(X) - window_len + i].view(-1, 1) for i in range(window_len)], dim=1)
y_train = X[window_len:]

In [21]:
# Correct initialization does not matter much in small models
Wemb = torch.randn((len(chars), 10), generator=generator, device=device_str)
W1 = torch.randn((10 * window_len, 100), generator=generator, device=device_str) * 0.01
b1 = torch.randn((100,), generator=generator, device=device_str) * 0.01
W2 = torch.randn((100, 100), generator=generator, device=device_str) * 0.01
b2 = torch.randn((100,), generator=generator, device=device_str) * 0.01
W3 = torch.randn((100, len(chars)), generator=generator, device=device_str) * 0.01

params = [W1, b1, W2, b2, W3, Wemb]
for p in params:
    p.requires_grad = True
sum(p.nelement() for p in params)

41980

In [22]:
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm

batch_size = 256
for i in tqdm(range(1000000)):
    inds = torch.randint(0, X_train.shape[0], (batch_size,), generator=generator, device=device_str)
    emb = Wemb[X_train[inds]]
    a = torch.tanh(torch.tanh(emb.view(batch_size, -1) @ W1 + b1) @ W2 + b2) @ W3
    loss = F.cross_entropy(a, y_train[inds])
    #ae = torch.exp(a)
    #se = torch.sum(ae, dim=1, keepdim=True)
    #res = ae / se
    #loss = -torch.sum(torch.log(res[np.linspace(0, batch_size - 1, num=batch_size, dtype=np.int32), y_train[inds]])) / batch_size
   
    for p in params:
        p.grad = None
    loss.backward()
    
    lr = 0.01 if i < 30000 else 0.001
    for p in params:
        p.data -= p.grad * lr
    if i % 100000 == 0:
        print(loss.item())
    #print(res[])
#Wemb[X_train[0]]

  0%|          | 211/1000000 [00:00<15:51, 1050.82it/s]

5.288217067718506


 10%|█         | 100143/1000000 [01:37<14:19, 1047.49it/s]

2.9073073863983154


 20%|██        | 200167/1000000 [03:14<12:32, 1063.27it/s]

...

 80%|████████  | 800182/1000000 [12:56<03:10, 1048.43it/s]

2.256413459777832


 90%|█████████ | 900201/1000000 [14:33<01:35, 1040.55it/s]

2.5296003818511963


100%|██████████| 1000000/1000000 [16:10<00:00, 1030.39it/s]


In [30]:
text = ['<s>'] + list('ماست لبنیات است')
X_test = []
print(''.join(text[:10]))
for c in text[:10]:
    X_test.append(ctoi[c])
X_test = torch.tensor(X_test)
with torch.no_grad():
    r = torch.tanh(Wemb[X_test].view(1, -1) @ W1 + b1) @ W2
    print(chars[torch.argmax(r)])

<s>ماست لبنی
ف
